In [1]:
import numpy as np
from scipy.stats import multivariate_normal
from matplotlib import pyplot as plt
from copy import copy
from datetime import datetime, timedelta

from stonesoup.functions import gm_reduce_single

from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.updater.kalman import KalmanUpdater
from stonesoup.predictor.imm import IMMPredictor
from stonesoup.updater.imm import IMMUpdater
from stonesoup.models.transition.linear import ConstantVelocity, \
    CombinedLinearGaussianTransitionModel, LinearGaussianTimeInvariantTransitionModel, RandomWalk

from stonesoup.models.measurement.linear import LinearGaussian

from stonesoup.types.state import GaussianState
from stonesoup.types.hypothesis import SingleHypothesis
from stonesoup.types.detection import Detection
from stonesoup.types.state import StateVector, CovarianceMatrix, \
    GaussianMixtureState, WeightedGaussianState
from stonesoup.types.track import Track

from stonesoup.simulator.simple import SingleTargetGroundTruthSimulator

from matplotlib.patches import Ellipse


In [2]:
from bayou.datastructures import Gaussian as EMGPB2Gaussian
from bayou.datastructures import GMM as EMGPB2GMM
from bayou.datastructures import GMMSequence as EMGPB2GMMSequence
from bayou.models import LinearModel as EMGPB2LinearModel
from bayou.models import ConstantVelocity as EMGPB2ConstantVelocity
from bayou.expmax.skf import SKF

Plot function

In [3]:
def plot_cov_ellipse(cov, pos, nstd=2, ax=None, **kwargs):
    """
    Plots an `nstd` sigma error ellipse based on the specified covariance
    matrix (`cov`). Additional keyword arguments are passed on to the
    ellipse patch artist.
    Parameters
    ----------
        cov : The 2x2 covariance matrix to base the ellipse on
        pos : The location of the center of the ellipse. Expects a 2-element
            sequence of [x0, y0].
        nstd : The radius of the ellipse in numbers of standard deviations.
            Defaults to 2 standard deviations.
        ax : The axis that the ellipse will be plotted on. Defaults to the
            current axis.
        Additional keyword arguments are pass on to the ellipse patch.
    Returns
    -------
        A matplotlib ellipse artist
    """

    def eigsorted(cov):
        vals, vecs = np.linalg.eigh(cov)
        order = vals.argsort()[::-1]
        return vals[order], vecs[:, order]

    if ax is None:
        ax = plt.gca()

    vals, vecs = eigsorted(cov)
    theta = np.degrees(np.arctan2(*vecs[:, 0][::-1]))

    # Width and height are "full" widths, not radius
    width, height = 2 * nstd * np.sqrt(vals)
    ellip = Ellipse(xy=pos, width=width, height=height, angle=theta,
                    **kwargs)

    ax.add_artist(ellip)
    return ellip


Groundtruth and detections

In [62]:
gt_track = []
gt_time = []
iter_model_1 = 50
iter_model_2 = 50

gt_transition_model_1 = CombinedLinearGaussianTransitionModel(
                        (RandomWalk(0.5 ** 2),
                        RandomWalk(0.5 ** 2))
                        )

timestamp_init = datetime.now()
state_init_1 = GaussianState(StateVector([[0], [0]]),
                           CovarianceMatrix(np.diag([0.0, 0.0])),
                           timestamp=timestamp_init)
gt_generator_1 = SingleTargetGroundTruthSimulator(gt_transition_model_1, state_init_1, number_steps=iter_model_1)

for time, gnd_paths in gt_generator_1.groundtruth_paths_gen():
    gnd_path = gnd_paths.pop()
    gt_track.append(gnd_path.state)
    gt_time.append(time)

gt_transition_model_2 = CombinedLinearGaussianTransitionModel(
                        (RandomWalk(5.0 ** 2),
                        RandomWalk(5.0 ** 2))
                        )
state_init_2 = GaussianState(gt_track[-1].state_vector,
                           CovarianceMatrix(np.diag([0.0, 0.0])),
                           timestamp=gt_time[-1])
gt_generator_2 = SingleTargetGroundTruthSimulator(gt_transition_model_2, state_init_2, number_steps=iter_model_2)
for time, gnd_paths in gt_generator_2.groundtruth_paths_gen():
    gnd_path = gnd_paths.pop()
    gt_track.append(gnd_path.state)
    gt_time.append(time)

detection_track = []
mesurement_noise_1 = np.diag([1.0 ** 2, 1.0 ** 2])
measurement_model_1 = LinearGaussian(ndim_state=2, mapping=[0, 1], noise_covar=mesurement_noise_1)
for i in range(iter_model_1):
    m_ = Detection(measurement_model_1.function(gt_track[i].state_vector, measurement_model_1.rvs(1)), gt_time[i])
    detection_track.append(m_)

mesurement_noise_2 = np.diag([0.25 ** 2, 0.25 ** 2])
measurement_model_2 = LinearGaussian(ndim_state=2, mapping=[0, 1], noise_covar=mesurement_noise_2)
for i in range(iter_model_2):
    ii = i + iter_model_1
    m_ = Detection(measurement_model_2.function(gt_track[ii].state_vector, measurement_model_2.rvs(1)), gt_time[ii])
    detection_track.append(m_)

# print(gt_track)
# print(detection_track)
# print(gt_time)

Filter model. Testing without EMGPB2 estimated parameters.

In [66]:
ft_transition_model_1 = CombinedLinearGaussianTransitionModel(
                        (RandomWalk(1.5 ** 2),
                        RandomWalk(1.5 ** 2))
                        )
ft_transition_model_2 = CombinedLinearGaussianTransitionModel(
                        (RandomWalk(3.0 ** 2),
                        RandomWalk(3.0 ** 2))
                        )
ft_measurement_noise = (1.0 **2) * np.eye(2)

ft_measurement_model = LinearGaussian(ndim_state=2, mapping=[0, 1],
                                   noise_covar=ft_measurement_noise)

ft_model_transition_matrix = np.array([[0.5, 0.5],
                                       [0.5, 0.5]])
ft_predictor_1 = KalmanPredictor(ft_transition_model_1)
ft_predictor_2 = KalmanPredictor(ft_transition_model_2)
ft_imm_predictor = IMMPredictor([ft_predictor_1, ft_predictor_2], ft_model_transition_matrix)

ft_updater = KalmanUpdater(ft_measurement_model)
ft_imm_updater = IMMUpdater([ft_updater, ft_updater], ft_model_transition_matrix)

ft_state_init = WeightedGaussianState(StateVector([[0], [0]]),
                                      CovarianceMatrix(np.diag([5.0, 5.0])),
                                      timestamp=timestamp_init,
                                      weight=0.5)

prior1 = copy(ft_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)):
    measurement = detection_track[i]
    # State prediction
    prediction = ft_imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = ft_imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = ft_imm_updater.update(hyp)
    track.append(prior)
    track_error.append(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))) - ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 0.9882903659241924


Filter model. Testing with correct parameters.

In [65]:
model_transition_matrix = np.array([[0.3, 0.7],
                                    [0.3, 0.7]])

predictor_1 = KalmanPredictor(gt_transition_model_1)
predictor_2 = KalmanPredictor(gt_transition_model_2)
imm_predictor = IMMPredictor([predictor_1, predictor_2],
                             model_transition_matrix)

updater_1 = KalmanUpdater(measurement_model_1)
updater_2 = KalmanUpdater(measurement_model_2)
imm_updater = IMMUpdater([updater_1, updater_2], model_transition_matrix)

prior1 = copy(ft_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)):
    measurement = detection_track[i]
    # State prediction
    prediction = imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = imm_updater.update(hyp)
    track.append(prior)
    track_error.append(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))) - ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    # print(ft_measurement_model.function(track[i].state_vector, np.zeros((2,1))))
    # print(ft_measurement_model.function(gt_track[i].state_vector, np.zeros((2,1))))
    # print('----------------------')
    
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 0.8357635113594671


EMGPB2

In [37]:
F = np.asarray([
        [1, 0],
        [0, 1],
    ])
H = np.asanyarray([
    [1, 0],
    [0, 1]
])
def get_Q(Q_sig):
    Q = (Q_sig ** 2) * np.diag([1.0, 1.0])
    return Q
def transfer_Q(Q):
    new_Q = np.zeros_like(Q)
    for i in range(new_Q.shape[0]):
        new_Q[i, i] = Q[i, i]
    return new_Q

In [67]:
g1 = EMGPB2Gaussian(np.zeros([2, 1]), 10.0*np.eye(2))
g2 = EMGPB2Gaussian(np.zeros([2, 1]), 10.0*np.eye(2))
initial_gmm_state = EMGPB2GMM([g1, g2])

# measurements = 5 * np.random.randn(200, 2, 1) + 1

detection_track_EMGPB2 = [ele.state_vector for ele in detection_track]
gmmsequence = EMGPB2GMMSequence(np.float64(detection_track_EMGPB2), initial_gmm_state)

m1 = EMGPB2LinearModel(F, get_Q(1.5), H, (1.0 ** 2)*np.eye(2))
m2 = EMGPB2LinearModel(F, get_Q(3.0), H, (1.0 ** 2)*np.eye(2))
initial_models = [m1, m2]

Z = np.array([[0.5, 0.5], 
              [0.5, 0.5]])
dataset = [gmmsequence]

new_models, Z, dataset, LL = SKF.EM(dataset, initial_models, Z,
                                    max_iters=100, threshold=0.0001, learn_H=False, learn_R=True,
                                    learn_A=True, learn_Q=True, learn_init_state=False, learn_Z=True,
                                    keep_Q_structure=False, diagonal_Q=False, wishart_prior=False)
new_models[0].Q = (new_models[0].Q + new_models[0].Q.T)/2
new_models[1].Q = (new_models[1].Q + new_models[1].Q.T)/2

-539.2202190242415
model -- 0 new_A: 
[[ 0.99040152 -0.0108737 ]
 [-0.01688227  0.97240568]]
model -- 1 new_A: 
[[ 0.87926508  0.00528452]
 [-0.08426052  0.96746978]]
model -- 0 new_Q: 
[[ 2.87541486 -0.01484501]
 [-0.01479762  2.67369307]]
model -- 1 new_Q: 
[[16.84137549  0.86952608]
 [ 0.86962246 13.59771055]]
model -- 0 new_R: 
[[0.93503406 0.01960437]
 [0.01960437 0.86877308]]
model -- 1 new_R: 
[[ 1.26665191 -0.00362093]
 [-0.00362093  1.0530225 ]]
new_Z: 
[[0.45195172 0.53794727]
 [0.45195172 0.53794727]]
-----------------------------------------------
-520.4006030138028
model -- 0 new_A: 
[[ 1.00343561 -0.02096495]
 [-0.01378028  0.95665151]]
model -- 1 new_A: 
[[ 0.83308589  0.00407718]
 [-0.1010589   0.96841367]]
model -- 0 new_Q: 
[[ 3.34759656 -0.02053513]
 [-0.01596177  2.85869126]]
model -- 1 new_Q: 
[[19.84316537  0.96430993]
 [ 0.97157594 15.33789696]]
model -- 0 new_R: 
[[0.95686345 0.01889871]
 [0.01889857 0.81087846]]
model -- 1 new_R: 
[[ 1.23774872 -0.01941819]
 [-

model -- 1 new_R: 
[[ 1.02076971 -0.09738922]
 [-0.0973441   0.66639123]]
new_Z: 
[[0.60079554 0.38910345]
 [0.60079554 0.38910345]]
-----------------------------------------------
-508.30906601041795
model -- 0 new_A: 
[[ 1.04436412 -0.04045615]
 [-0.050129    0.85064716]]
model -- 1 new_A: 
[[ 0.71106168  0.01470784]
 [-0.12606479  1.01559904]]
model -- 0 new_Q: 
[[ 3.2206778  -0.30918212]
 [-0.30804855  1.96892097]]
model -- 1 new_Q: 
[[23.41548028 -0.83396057]
 [-0.82932739 18.4138586 ]]
model -- 0 new_R: 
[[ 0.77064291 -0.03409087]
 [-0.034071    0.46215224]]
model -- 1 new_R: 
[[ 1.01721525 -0.09919329]
 [-0.09916045  0.64458116]]
new_Z: 
[[0.597617   0.39228199]
 [0.597617   0.39228199]]
-----------------------------------------------
-507.81570054461537
model -- 0 new_A: 
[[ 1.04556575 -0.03910121]
 [-0.05469288  0.84482142]]
model -- 1 new_A: 
[[ 0.71206242  0.01403626]
 [-0.12394261  1.01664893]]
model -- 0 new_Q: 
[[ 3.18005733 -0.29829279]
 [-0.29640567  1.8959776 ]]
model 

model -- 1 new_A: 
[[ 0.7205064   0.00888763]
 [-0.1111002   1.02049631]]
model -- 0 new_Q: 
[[ 2.9041692  -0.11210344]
 [-0.10182606  1.21543622]]
model -- 1 new_Q: 
[[22.7195015  -0.45502181]
 [-0.45109427 18.04714547]]
model -- 0 new_R: 
[[ 0.75630912 -0.03544725]
 [-0.03546594  0.30014613]]
model -- 1 new_R: 
[[ 1.0663143  -0.11581911]
 [-0.11618754  0.41984199]]
new_Z: 
[[0.57216167 0.41773732]
 [0.57216167 0.41773732]]
-----------------------------------------------
-502.5133781081115
model -- 0 new_A: 
[[ 1.0506508  -0.03110812]
 [-0.0898788   0.79864759]]
model -- 1 new_A: 
[[ 0.72081389  0.008873  ]
 [-0.11110093  1.02044243]]
model -- 0 new_Q: 
[[ 2.90040229 -0.10899842]
 [-0.09883553  1.21181578]]
model -- 1 new_Q: 
[[22.70513552 -0.45367501]
 [-0.44983071 18.05898911]]
model -- 0 new_R: 
[[ 0.75825749 -0.03627005]
 [-0.03629549  0.29463786]]
model -- 1 new_R: 
[[ 1.07190179 -0.11704166]
 [-0.11741852  0.41020586]]
new_Z: 
[[0.571972   0.41792699]
 [0.571972   0.41792699]]
-

Filter model. Testing with learnt parameters from EMGPB2.

In [68]:
# em_transition_model_1 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[0].A, covariance_matrix=transfer_Q(new_models[0].Q))
# em_transition_model_2 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[1].A, covariance_matrix=transfer_Q(new_models[1].Q))

em_transition_model_1 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[0].A, covariance_matrix=new_models[0].Q)
em_transition_model_2 = LinearGaussianTimeInvariantTransitionModel(transition_matrix=new_models[1].A, covariance_matrix=new_models[1].Q)
em_measurement_noise_1 = new_models[0].R
em_measurement_noise_2 = new_models[1].R

em_measurement_model_1 = LinearGaussian(ndim_state=2, mapping=[0, 1],
                                   noise_covar=em_measurement_noise_1)
em_measurement_model_2 = LinearGaussian(ndim_state=2, mapping=[0, 1],
                                   noise_covar=em_measurement_noise_2)

em_model_transition_matrix = np.array([[0.5, 0.5],
                                       [0.5, 0.5]])
em_predictor_1 = KalmanPredictor(em_transition_model_1)
em_predictor_2 = KalmanPredictor(em_transition_model_2)
em_imm_predictor = IMMPredictor([em_predictor_1, em_predictor_2], em_model_transition_matrix)

em_updater_1 = KalmanUpdater(em_measurement_model_1)
em_updater_2 = KalmanUpdater(em_measurement_model_2)
em_imm_updater = IMMUpdater([em_updater_1, em_updater_2], em_model_transition_matrix)

em_state_init = WeightedGaussianState(StateVector([[0], [0]]),
                                      CovarianceMatrix(np.diag([5.0, 5.0])),
                                      timestamp=timestamp_init,
                                      weight=0.5)

prior1 = copy(em_state_init)
prior = GaussianMixtureState([prior1, prior1])
track = Track([copy(prior)])
track_error = []

for i in range(1, len(detection_track)):
    measurement = detection_track[i]
    
    # State prediction
    prediction = em_imm_predictor.predict(track.state, timestamp=gt_time[i])
    # Measurement prediction
    meas_prediction = em_imm_updater.predict_measurement(prediction)
    # State update
    hyp = SingleHypothesis(prediction, measurement)
    prior = em_imm_updater.update(hyp)
    track.append(prior)
    track_error.append(em_measurement_model_1.function(track[i].state_vector, np.zeros((2,1))) - em_measurement_model_1.function(gt_track[i].state_vector, np.zeros((2,1))))
    
    '''
    fig, (ax1, ax2) = plt.subplots(2,1)
    # Generate plots
    ax1.cla()
    ax2.cla()
    # PLot true trajectory
    data = np.array([state.state_vector for state in gt_track])
    ax1.plot(data[:, 0], data[:, 2], 'b-')
    # PLot estimated trajectory
    data = np.array([state.state_vector for state in track.states])
    ax1.plot(data[:, 0], data[:, 2], 'r-')
    # Plot innovation covariance
    plot_cov_ellipse(meas_prediction.covar,
                     meas_prediction.mean, edgecolor='b',
                     facecolor='none', ax=ax1)
    # Plot estimated covariance
    plot_cov_ellipse(track.state.covar[[0, 2], :][:, [0, 2]],
                     track.state.mean[[0, 2], :], edgecolor='r',
                     facecolor='none', ax=ax1)
    # Visualise model weights
    ax2.bar([1,2], prior.weights.ravel())
    plt.pause(0.0001)
    '''

track_error = np.asarray(track_error)
track_error = np.squeeze(track_error)
rmse_tmp = []
for track_error_ in track_error:
    rmse_tmp.append(track_error_ @ track_error_.T)
rmse = np.sqrt(np.mean(np.array(rmse_tmp)))
print("RMSE: " + str(rmse))

RMSE: 0.9037775850790563


In [10]:
len(rmse_tmp)

279